In [1]:
import json
import numpy as np
import csv
import pymongo as pm

In [2]:
google_img_str = 'http://drive.google.com/uc?export=view&id='

In [3]:
data_dict = {}
line_counter = 0
with open('img_ids.csv') as csv_file:
    for line in csv_file:
        if line_counter == 0:
            header_list = line.strip().split(',')
        else:
            line_data = line.strip().split(',')
            data_dict[line_data[0]] = {i: j for (i, j) in zip(header_list[1:], line_data[1:])}
        line_counter += 1
print(header_list)

['Name', 'Date', 'Size', 'URL', 'ID']


In [4]:
img_types = [i for i in set([i.split('_')[0] for i in data_dict.keys()]) if '.' not in i]

In [41]:
json_list = []
for img_type in img_types:
    if len(img_type) > 1:
        png_files = [i for i in data_dict.keys() if (img_type in i) & ('.png' in i)]
        compressed_files = [i for i in png_files if ('bpg' in i) or ('final' in i)]
        original_files = [i for i in png_files if ('original') in i or (('pxh' in i) & ('bpg' not in i))]
        if len(original_files) > 1:
            original_file = [i for i in original_files if 'px' in i][0]
        else:
            original_file = original_files[0]
        for compressed_file in compressed_files:
            if 'final' in compressed_file:
                image_height = 'NA'
                compression_level ='human'
                compression_mode = 'human'
            else:
                image_height = compressed_file.split('_')[1].split('pxh')[0]
                compression_level = compressed_file.split('_bpg')[1].split('.')[0]
                compression_mode = 'bpg'
            json_list.append(
                {
                    'image_filename' : compressed_file,
                    'image_name': compressed_file.split('_')[0],
                    'image_height': image_height,
                    'compression_level': compression_level,
                    'compression_mode': compression_mode,
                    'orig_url': google_img_str + data_dict[original_file]['ID'],
                    'compressed_url' : google_img_str + data_dict[compressed_file]['ID']
                }
            )

In [42]:
json_list[0]

{'compressed_url': 'http://drive.google.com/uc?export=view&id=1zFbvwgb4uLHLzilqgvsIdrn2TQXm6SQ_',
 'compression_level': '43',
 'compression_mode': 'bpg',
 'image_filename': 'rockwall_1029pxh_bpg43.png',
 'image_height': '1029',
 'image_name': 'rockwall',
 'orig_url': 'http://drive.google.com/uc?export=view&id=1svOMbWnGZo8SsWb4_2wgaKuDbPovLEF-'}

In [7]:
image_id = compressed_file
image_name = compressed_file.split('_')[0]
image_size = compressed_file.split('_')[1]
compression_level = compressed_file.split('_')[2]

In [8]:
image_size.split('pxh')[0]

'440'

In [43]:
coll.distinct('compression_mode')

[]

In [9]:
json_list[0]['img_url']

'http://drive.google.com/uc?export=view&id=1zFbvwgb4uLHLzilqgvsIdrn2TQXm6SQ_'

In [10]:
# with open('test.json', 'w') as writefile:
#     json.dump(json_list, writefile)

In [11]:
pswd = 'popeye2mongo'
user = 'ihwang'
host = 'popeye2'

conn = pm.MongoClient('mongodb://' + user + ':' + pswd + '@127.0.0.1')

In [12]:
conn.database_names()

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: database_names is deprecated. Use list_database_names instead.
  """Entry point for launching an IPython kernel.


[u'admin',
 u'config',
 u'highschoolers',
 u'kiddraw',
 u'local',
 u'stimuli',
 u'testDatabase']

In [45]:
db = conn['stimuli']
coll = db['bpg_hc_eval'] 

In [46]:
for i in json_list:
    coll.insert_one(i)

In [47]:
coll.count()

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


78

In [48]:
coll.find_one()

{u'_id': ObjectId('5ca3fc64b344ba55de40525c'),
 u'compressed_url': u'http://drive.google.com/uc?export=view&id=1zFbvwgb4uLHLzilqgvsIdrn2TQXm6SQ_',
 u'compression_level': u'43',
 u'compression_mode': u'bpg',
 u'image_filename': u'rockwall_1029pxh_bpg43.png',
 u'image_height': u'1029',
 u'image_name': u'rockwall',
 u'orig_url': u'http://drive.google.com/uc?export=view&id=1svOMbWnGZo8SsWb4_2wgaKuDbPovLEF-'}

In [23]:
coll.count()

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


78

In [50]:
coll.distinct('compression_level')

[u'43',
 u'39',
 u'34',
 u'human',
 u'49',
 u'46',
 u'48',
 u'30',
 u'35',
 u'44',
 u'28',
 u'31',
 u'38',
 u'24',
 u'51',
 u'37',
 u'29',
 u'50',
 u'33',
 u'45',
 u'23',
 u'40',
 u'41',
 u'27',
 u'36',
 u'19',
 u'42',
 u'47',
 u'26',
 u'25']